In [1]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

In [3]:
def generator():
    while True:
        yield

In [11]:
i = 0
mul = 100
all_a = []
for _ in tqdm(generator()):
    r = requests.get(f'https://oarep.usim.edu.my/jspui/simple-search?query=&location=publications&sort_by=score&order=desc&rpp={mul}&etal=0&start={i * mul}',
                verify = False)
    soup = BeautifulSoup(r._content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a]
    a = [a_ for a_ in a if a and 'jspui/handle/' in a_]
    if not len(a):
        break
    all_a.extend(a)
    i += 1

118it [03:25,  1.74s/it]


In [12]:
set(all_a)

{'/jspui/handle/123456789/11893',
 '/jspui/handle/123456789/1110',
 '/jspui/handle/123456789/2976',
 '/jspui/handle/123456789/17642',
 '/jspui/handle/123456789/19856',
 '/jspui/handle/123456789/20048',
 '/jspui/handle/123456789/17992',
 '/jspui/handle/123456789/7637',
 '/jspui/handle/123456789/10760',
 '/jspui/handle/123456789/10328',
 '/jspui/handle/123456789/5619',
 '/jspui/handle/123456789/3876',
 '/jspui/handle/123456789/5007',
 '/jspui/handle/123456789/13348',
 '/jspui/handle/123456789/12916',
 '/jspui/handle/123456789/12853',
 '/jspui/handle/123456789/11595',
 '/jspui/handle/123456789/12974',
 '/jspui/handle/123456789/17110',
 '/jspui/handle/123456789/5102',
 '/jspui/handle/123456789/13762',
 '/jspui/handle/123456789/4757',
 '/jspui/handle/123456789/3555',
 '/jspui/handle/123456789/3511',
 '/jspui/handle/123456789/19404',
 '/jspui/handle/123456789/2392',
 '/jspui/handle/123456789/8596',
 '/jspui/handle/123456789/3660',
 '/jspui/handle/123456789/4693',
 '/jspui/handle/123456789/87

In [13]:
len(all_a)

11776

In [14]:
import json

with open('all_a-usim.json', 'w') as fopen:
    json.dump(list(set(all_a)), fopen)

In [18]:
all_a = list(set(all_a))
all_a = ['https://oarep.usim.edu.my/' + a for a in all_a]

In [21]:
def get_pdf(url):
    try:
        r = requests.get(url, timeout = 10.0, verify = False)
        soup = BeautifulSoup(r._content, "lxml")
        a = soup.find_all('a')
        a = [a_.get('href') for a_ in a]
        a = [a_ for a_ in a if a_ and '.pdf' in a_ and 'haspreview' not in a_]
        a = list(set(a))
        return a
    except Exception as e:
        print(e)
        return None

get_pdf(all_a[0])

['/jspui/bitstream/123456789/11893/1/The_Process_of_Transformational_Leadership_in_Achieving_High_Performance_Workforce_that_Exceeds.pdf']

In [22]:
!rm -rf pages-usim
!mkdir pages-usim

In [25]:
import os
from concurrent.futures import ThreadPoolExecutor, as_completed

max_worker = 10

all_a = sorted(all_a)
for i in tqdm(range(0, len(all_a), max_worker)):
    
    filename = os.path.join('pages-usim', f'{i}.json')
    
    if os.path.exists(filename) and os.path.getsize(filename) > 1000:
        continue
    
    pdfs = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_pdf, url): url for url in all_a[i: i + max_worker]}

        for future in as_completed(futures):
            r = future.result()
            if r:
                pdfs.extend(r)
    
    with open(filename, 'w') as fopen:
        json.dump(pdfs, fopen)

100%|███████████████████████████████████████| 1178/1178 [05:40<00:00,  3.46it/s]
